In [2]:
import os
import glob
import argparse
import matplotlib

# Keras / TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'
from keras.models import load_model
from layers import BilinearUpSampling2D
from utils import predict, load_images, display_images
from matplotlib import pyplot as plt
import skimage
from skimage.transform import resize

import numpy as np
import cv2
from PIL import Image


Using TensorFlow backend.
d:\anaconda3\envs\tensflow1x\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\anaconda3\envs\tensflow1x\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\anaconda3\envs\tensflow1x\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\anaconda3\envs\tensflow1x\lib\site-packages\tensorflow\python\fra

In [10]:
# Argument Parser
parser = argparse.ArgumentParser(description='High Quality Monocular Depth Estimation via Transfer Learning')
parser.add_argument('--model', default='nyu.h5', type=str, help='Trained Keras model file.')
parser.add_argument('--input', default='examples/*.png', type=str, help='Input filename or folder.')
args,unknown = parser.parse_known_args()

In [4]:
# Custom object needed for inference and training
#双线性采样
custom_objects = {'BilinearUpSampling2D': BilinearUpSampling2D, 'depth_loss_function': None}

print('Loading model...')

# Load model into GPU / CPU
model = load_model(args.model, custom_objects=custom_objects, compile=False)

print('\nModel loaded ({0}).'.format(args.model))

Loading model...
Instructions for updating:
Colocations handled automatically by placer.

Model loaded (nyu.h5).


In [11]:
# Input images
inputs = load_images( glob.glob(args.input) )
print('\nLoaded ({0}) images of size {1}.'.format(inputs.shape[0], inputs.shape[1:]))

# Compute results
outputs = predict(model, inputs)
print(outputs.shape)


Loaded (1) images of size (480, 640, 3).
(1, 240, 320, 1)


In [6]:
from skimage import io
def corp_margin(img):
        img2=img.sum(axis=2)
        (row,col)=img2.shape
        row_top=0
        raw_down=0
        col_top=0
        col_down=0
        for r in range(0,row):
                if img2.sum(axis=1)[r]<700*col:
                        row_top=r
                        break
 
        for r in range(row-1,0,-1):
                if img2.sum(axis=1)[r]<700*col:
                        raw_down=r
                        break
 
        for c in range(0,col):
                if img2.sum(axis=0)[c]<700*row:
                        col_top=c
                        break
 
        for c in range(col-1,0,-1):
                if img2.sum(axis=0)[c]<700*row:
                        col_down=c
                        break
 
        new_img=img[row_top:raw_down+1,col_top:col_down+1,0:3]
        return new_img

# viz1 = display_images(outputs.copy())
# img_re = corp_margin(viz1)
# io.imsave('result.png',img_re)
# io.imshow(img_re)


In [12]:
#将推理图像还原为尺寸640*480*3的png图像
#matplotlib problem on ubuntu terminal fix
#matplotlib.use('TkAgg')   

# Display results
print(type(outputs),outputs.shape)
# img = outputs[0][:,:,0]
# print(img.shape)
# viz = display_images(outputs.copy(), inputs.copy())
viz1 = display_images(outputs.copy())
viz1_re = corp_margin(viz1)
print(type(viz1_re))
print(viz1_re.shape)
# viz1_re = viz1_re.resize((480, 640), Image.ANTIALIAS)
viz1_re = cv2.resize(viz1_re, dsize=(640,480), interpolation=cv2.INTER_LINEAR)
print("修改后尺寸：",viz1_re.shape)
#cv2
# print(type(viz1_re))
# print(viz1_re.shape)
# # viz1_re = cv2.cvtColor(viz1_re, cv2.COLOR_BGR2RGB)

# cv2.imshow('input_image', viz1)
# cv2.imwrite('./realPics_t3/1_image_depth.png', viz1)

# print(type(viz1))
# print(viz1.shape)
# #img = outputs
# # img = outputs.copy()
# # plt.figure(figsize=(10,5))
# plt.axis('off')# 不显示坐标轴
# plt.margins(0,0)
# plt.subplots_adjust(top=1,bottom=0,left=0,right=1,hspace=0,wspace=0)
# plt.imshow(viz1_re)
# plt.savefig('./realPics_t3/5_image_depth.png')
# plt.show()

io.imsave('./paperpic/pic7_plasma_depth.png',viz1_re)
# io.imshow(viz1_re)


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


<class 'numpy.ndarray'> (1, 240, 320, 1)
<class 'numpy.ndarray'>
(240, 320, 3)
修改后尺寸： (480, 640, 3)


In [9]:
#图像大小调整
import cv2

# 改变图像大小
import cv2
im1 = cv2.imread('./examples/pic7.png')
im2 = cv2.resize(im1,(640,480),)  # 为图片重新指定尺寸
cv2.imwrite('./examples/pic7.png',im2)


True

In [13]:
#灰度图伪彩色上色
import cv2
import numpy as np
def turn_plasma(img):
    for i in range(480):
        for j in range(640):
            img[i,j] = 255 - img[i,j]
    return img

#gray = cv2.imread('./paperpic/pic1_1.png',0)  # 读取灰度图像
img_bgr = cv2.imread('./paperpic/pic7_plasma_depth.png')
gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
print(img_bgr)
#cv2.imwrite('./paperpic/pic1_plasma_depth1.png',iFColor)
# cv2.imshow('iFColor',gray)
# cv2.waitKey()

h, w = img_bgr.shape[:2]  # 图片的高度, 宽度
print(img_bgr.shape,h ,w)
#print(gray)
# for i in range(h):
#     for j in range(w):
#         if img_bgr[i,j,0] > 20:
#             img_bgr[i,j,0] = img_bgr[i,j,0] - 10
#         else :
#             img_bgr[i,j,0] = img_bgr[i,j,0]
#         if img_bgr[i,j,1] > 10:
#             img_bgr[i,j,1] = img_bgr[i,j,1] - 5
#         else :
#             img_bgr[i,j,1] = 15
#         if img_bgr[i,j,2] < 230:
#             img_bgr[i,j,2] = img_bgr[i,j,2] + 14
#         else :
#             img_bgr[i,j,2] = 255
##归一化
# ymax = 255
# ymin = 0
# xmax = max(map(max,gray))
# xmin = min(map(min,gray))
 
# for i in range(h):
#     for j in range(w):
#         gray[i][j] = round(((ymax-ymin)*(gray[i][j]-xmin)/(xmax-xmin))+ymin)

# print(gray.shape)

#gray = turn_plasma(gray)
iFColor = cv2.applyColorMap(gray, colormap=cv2.COLORMAP_JET) #标准标签热力图
#iFColor = cv2.applyColorMap(gray, colormap=cv2.COLORMAP_PLASMA) #标准标签  黄紫
#iFColor = cv2.applyColorMap(gray, colormap=cv2.COLORMAP_PLASMA) #标准标签
cv2.imwrite('./paperpic/pic7_jet_depth.png',iFColor)
cv2.imshow('iFColor',iFColor)
cv2.waitKey()


[[[ 83 121 237]
  [ 89 112 231]
  [102  95 220]
  ...
  [147  38 172]
  [142  44 179]
  [139  48 182]]

 [[ 84 119 236]
  [ 90 111 231]
  [101  95 221]
  ...
  [148  37 171]
  [144  42 177]
  [141  45 180]]

 [[ 87 115 234]
  [ 91 109 230]
  [100  97 222]
  ...
  [149  35 169]
  [147  38 173]
  [146  40 175]]

 ...

 [[168   0 105]
  [168   0 105]
  [168   0 105]
  ...
  [166   1  95]
  [167   1 102]
  [167   0 105]]

 [[168   1 112]
  [168   1 111]
  [168   0 110]
  ...
  [167   1 100]
  [168   1 109]
  [168   1 113]]

 [[168   1 116]
  [168   1 115]
  [168   0 112]
  ...
  [167   1 102]
  [168   1 112]
  [168   1 117]]]
(480, 640, 3) 480 640


-1

In [1]:
#灰度图深度图
import cv2
gray = cv2.imread('./realPics_t3/499_image_depth.png',0)  # 读取灰度图像
cv2.imwrite('./realPics_t3/499_image_depthgray.png',gray)
cv2.imshow('iFColor',gray)
cv2.waitKey()


-1

In [1]:
#深度图与分割图融合
import cv2
img_seg = cv2.imread("./paperpic/pic7_6.png",cv2.IMREAD_UNCHANGED)
#print(img_seg)
print(img_seg.shape)
print(type(img_seg))
print(img_seg[0,0,0])

(480, 640, 4)
<class 'numpy.ndarray'>
46


In [2]:
img_depth = cv2.imread("./paperpic/pic7_jet_depth.png")
#print(img_depth)
print(img_depth.shape)
print(type(img_depth))

(480, 640, 3)
<class 'numpy.ndarray'>


In [4]:
# img_seg = cv2.imread("./realPics_t3/ransacTest_image_seg.jpg")
# print(img_seg)
# print(img_seg.shape)
"""
白色rgb 255,255,255
rgba 透明a=0
cv2读取 bgr
"""
img_ultimate = img_depth.copy()
i_u, j_u, k_u = img_ultimate.shape
print(i_u,j_u,k_u)
for i in range(i_u):
    for j in range(j_u):
        if img_seg[i,j,3] == 0:
            img_ultimate[i,j,0] = 255
            img_ultimate[i,j,1] = 255
            img_ultimate[i,j,2] = 255
        else:
            img_ultimate[i,j,0] = img_depth[i,j,0]
            img_ultimate[i,j,1] = img_depth[i,j,1]
            img_ultimate[i,j,2] = img_depth[i,j,2]

#img_ultimate = cv2.cvtColor(img_ultimate, cv2.COLOR_BGR2RGB)
cv2.imwrite('./paperpic/pic7_ulti.jpg',img_ultimate)


480 640 3


True


##下面不不予理睬

In [5]:
#图片转为.bin二进制数据
import io
import base64
from PIL import Image
import struct
import os
import pickle, glob

def image2byte(image):
    '''
    图片转byte
    image: 必须是PIL格式
    image_bytes: 二进制
    '''
    # 创建一个字节流管道
    img_bytes = io.BytesIO()
    # #把PNG格式转换成的四通道转成RGB的三通道，然后再保存成jpg格式
    # image = image.convert("RGB")
    # 将图片数据存入字节流管道， format可以按照具体文件的格式填写
    image.save(img_bytes, format="JPEG")
    # 从字节流管道中获取二进制
    image_bytes = img_bytes.getvalue()
    return image_bytes

def byte2image(byte_data):
    '''
    byte转为图片
    byte_data: 二进制
    '''
    image = Image.open(io.BytesIO(byte_data))
    return image


img_path = './realPics_t3/ransacTest_image.png'
img1 = Image.open(img_path)
img_bin = image2byte(img1)
print(type(img_bin),len(img_bin))

#写入bin文件
binname = './realPics_t3/ransacTest_image_py.bin'
binfile = open(binname , 'rb+') #打开bin文件
pickle.dump(img_bin,binfile) #写入datafarme


<class 'bytes'> 16729


FileNotFoundError: [Errno 2] No such file or directory: './realPics_t3/ransacTest_image_py.bin'

In [9]:
#coding=utf-8
 
'''
1-将图片转化为数组并存为二进制文件
2-从二进制文件中读取数并重新恢复为图片
'''
 
from __future__ import print_function
import numpy
import PIL.Image 
import pickle
import matplotlib.pyplot
import pdb
 
class Operation(object):
	image_base_path = "./rgb2bin/"
	data_base_path = "./bin2rgb/"
 
	def image_to_array(self,filenames):
		"""
		将图片转化为数组并存为二进制文件
		"""
		n = filenames.__len__()#获取图片个数
		result = numpy.array([]) #创建一个空的一维数组
		print("开始将图片转化为数组")
		for i in range(n):
			image = PIL.Image.open(self.image_base_path+filenames[i])
			r,g,b = image.split() # rgb通道分离
			# 注意：下面一定要reshpae(1024)使其变为一维数组，否则拼接的数据会出现错误，导致无法恢复图片
			length = 640*480
			r_arr = numpy.array(r).reshape(length)
			g_arr = numpy.array(g).reshape(length)
			b_arr = numpy.array(b).reshape(length)
			# 行拼接；最终结果：共n行，一行length列，为一张图片的rgb值
			image_arr = numpy.concatenate((r_arr,g_arr,b_arr))
			result = numpy.concatenate((result,image_arr))
 
		result = result.reshape(n,3072) # 将一维数组转化为n行3072列的二维数组
		print("转化数组over，开始保存为文件")
		file_path = self.data_base_path + 'ransacTest_image_py.bin'
		with open(file_path,mode='wb') as f:
			pickle.dump(result,f)
		print("保存成功")
 
	# def array_to_image(self,filename):
	# 	'''
	# 	从二进制文件中读取数据并重新恢复为图片
	# 	'''
	# 	with open(self.data_base_path + filename,mode='rb') as f:
	# 		arr = pickle.load(f) #加载并反序列化数据
	# 	rows = arr.shape[0] #rows=5
	# 	#pdb.set_trace()
	# 	#print("rows:",rows)
	# 	arr = arr.reshape(rows,3,32,32)
	# 	print(arr)	#打印数组
	# 	for index in range(rows):
	# 		a = arr[index]
	# 		#得到RGB通道
	# 		r = PIL.Image.fromarray(a[0]).convert('L')
	# 		g = PIL.Image.fromarray(a[1]).convert('L')
	# 		b = PIL.Image.fromarray(a[2]).convert('L')
	# 		image = PIL.Image.merge("RGB",(r,g,b))
	# 		#显示图片
	# 		matplotlib.pyplot.imshow(image)
	# 		matplotlib.pyplot.show()
	# 		#image.save(self.image_base_path + "result" + str(index) + ".png",'png')
 
if __name__ == "__main__":
	my_operator = Operation()
	images = []
	for j in range(1):
		images.append(str(j) + ".png")
# #	my_operator.image_to_array(images)
# 	my_operator.array_to_image('ransacTest_image_py.bin')
 

In [11]:
#coding=utf-8
 
'''
1-将图片转化为数组并存为二进制文件
2-从二进制文件中读取数并重新恢复为图片
'''
 
from __future__ import print_function
import numpy
import PIL.Image 
import pickle
import matplotlib.pyplot
import pdb

def image_to_array(filenames):
		"""
		将图片转化为数组并存为二进制文件
		"""
		image_base_path = "./bin2rgb/"
		n = 1#获取图片个数
		result = numpy.array([]) #创建一个空的一维数组
		print("开始将图片转化为数组")
		for i in range(n):
			image = PIL.Image.open(image_base_path+filenames)
			r,g,b = image.split() # rgb通道分离
			# 注意：下面一定要reshpae(1024)使其变为一维数组，否则拼接的数据会出现错误，导致无法恢复图片
			length = 640*480
			r_arr = numpy.array(r).reshape(length)
			g_arr = numpy.array(g).reshape(length)
			b_arr = numpy.array(b).reshape(length)
			# 行拼接；最终结果：共n行，一行length列，为一张图片的rgb值
			image_arr = numpy.concatenate((r_arr,g_arr,b_arr))
			result = numpy.concatenate((result,image_arr))
 
		result = result.reshape(n,length*3) # 将一维数组转化为n行3072列的二维数组
		print("转化数组over，开始保存为文件")
		file_path = image_base_path + 'ransacTest_image_py.bin'
		with open(file_path,mode='wb') as f:
			pickle.dump(result,f)
		print("保存成功")

filename = "ransacTest_image.png"
image_to_array(filename)
	

开始将图片转化为数组
转化数组over，开始保存为文件
保存成功
